In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI( api_key=os.environ.get("OPENAI_API_KEY"))

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

NameError: name 'getpass' is not defined

In [3]:
load_dotenv()
print(os.environ.get("TAVILY_API_KEY"))

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


In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2,api_key=os.environ.get("TAVILY_API_KEY"),include_domains = ['www.healthyschoolrecipes.com'])
search_results = search.invoke("what are the ingredients recipe for Stromboli")

In [5]:
search_results

[{'title': 'Stromboli USDA',
  'url': 'https://healthyschoolrecipes.com/recipes/stromboli-usda/',
  'content': 'Ingredients ; Dried oregano, 1 teaspoon ; Dried marjoram, ¼ teaspoon ; Dried thyme, ¼ teaspoon ; Lite mozzarella cheese, sliced. (3 pounds, 2 ounces).',
  'score': 0.79580134},
 {'title': 'Vegetable Stromboli USDA',
  'url': 'https://healthyschoolrecipes.com/recipes/vegetable-stromboli-usda/',
  'content': 'Ingredients ; Vegetable oil, 1/4 cup ; Salt, 1 teaspoon ; Sugar, 2 tablespoons ; Dried basil, 1 tablespoons 1/4 teaspoon ; Dried oregano, 1',
  'score': 0.7636429}]

In [6]:
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o", model_provider="openai",api_key=os.environ.get("OPENAI_API_KEY"))
search = TavilySearchResults(max_results=2,include_domains = ['www.healthyschoolrecipes.com'])
tools = [search]

agent_executor = create_react_agent(model, tools)



In [7]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke(
    {"messages": [HumanMessage(content="what are the ingredients recipe for Healthy Pasta with Spicy Crab")]}
)
response["messages"]

[HumanMessage(content='what are the ingredients recipe for Healthy Pasta with Spicy Crab', additional_kwargs={}, response_metadata={}, id='b3f7a5fb-713b-4d0a-9c0c-f0baa58973a6'),
 AIMessage(content="Here's a recipe for Healthy Pasta with Spicy Crab:\n\n### Ingredients:\n- **Pasta**: Whole wheat spaghetti or linguine (as per serving size recommended)\n- **Spicy Crab Mixture**:\n  - 1 cup of fresh lump crab meat\n  - 1-2 tablespoons of olive oil\n  - 2 cloves garlic, minced\n  - 1 red chili, thinly sliced (adjust according to spice preference)\n  - 1 teaspoon of chili flakes (optional, for extra heat)\n  - Zest and juice of 1 lemon\n  - Salt and freshly ground black pepper to taste\n  - 1/4 cup of white wine (optional)\n- **Vegetables**: \n  - 1 cup cherry tomatoes, halved\n  - A handful of spinach or arugula\n  - 1/4 cup of chopped herbs (such as parsley or basil)\n- **Parmesan Cheese**: Grated, for topping (optional)\n\n### Instructions:\n1. **Cook the Pasta**: \n   - Cook the pasta in

In [11]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=
            """what are the ingredients recipe for Healthy Pasta with Spicy Crab
            example:
                 Ingredients:
                    - Whole wheat spaghetti or pasta of your choice: 8 ounces
                    - Olive oil**: 2 tablespoons
                    - Garlic**: 3 cloves, minced
                    - Red pepper flakes: 1 teaspoon (adjust to taste)
                    - Cooked crab meat: 8 ounces (fresh or canned)
                    - Cherry tomatoes: 1 cup, halved
                    - Lemon juice: from 1 lemon
                    - Lemon zest: from 1 lemon
                    - Fresh parsley: 1/4 cup, chopped
                    - Salt and black pepper: to taste
                    - Arugula or spinach: 2 cups
                    - Parmesan cheese (optional): for serving

            **note - make sure the response are in the format of the provided example**
            """)]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()



================================ Human Message =================================

what are the ingredients recipe for Healthy Pasta with Spicy Crab
            example:
                 Ingredients:
                    - Whole wheat spaghetti or pasta of your choice: 8 ounces
                    - Olive oil**: 2 tablespoons
                    - Garlic**: 3 cloves, minced
                    - Red pepper flakes: 1 teaspoon (adjust to taste)
                    - Cooked crab meat: 8 ounces (fresh or canned)
                    - Cherry tomatoes: 1 cup, halved
                    - Lemon juice: from 1 lemon
                    - Lemon zest: from 1 lemon
                    - Fresh parsley: 1/4 cup, chopped
                    - Salt and black pepper: to taste
                    - Arugula or spinach: 2 cups
                    - Parmesan cheese (optional): for serving

            **note - make sure the response are in the format of the provided example**
            
===================

In [12]:
tools = {
    "Click": "click",
    "Type": "type_text",
    "Scroll": "scroll",
    "Wait": "wait",
    "GoBack": "go_back",
    "Google": "to_google",
}


for node_name, tool in tools.items():
      print(f"{node_name," ",tool}" )

('Click', ' ', 'click')
('Type', ' ', 'type_text')
('Scroll', ' ', 'scroll')
('Wait', ' ', 'wait')
('GoBack', ' ', 'go_back')
('Google', ' ', 'to_google')
